<a href="https://colab.research.google.com/github/Tejas9523/EARLY-PREDICTION-OF-MENTAL-HEALTH-/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Function to load data
def load_data(data_dir, img_size=(48, 48)):
    data = []
    labels = []
    for emotion in os.listdir(data_dir):
        emotion_dir = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_dir) and len(os.listdir(emotion_dir)) > 0:
            for img_name in os.listdir(emotion_dir):
                img_path = os.path.join(emotion_dir, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    data.append(img)
                    labels.append(emotion)
                else:
                    print(f"Warning: Failed to load image: {img_path}")
        else:
            print(f"Warning: Empty or non-existent directory: {emotion_dir}")

    return np.array(data), np.array(labels)

In [ ]:
# Load training and testing data
X_train, y_train = load_data('/content/drive/MyDrive/projects/fer2013/train')
X_test, y_test = load_data('/content/drive/MyDrive/projects/fer2013/validation')

In [ ]:
# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Reshape data for the model
X_train = X_train.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

In [ ]:
# Convert labels to categorical
label_encoder = LabelEncoder()
all_labels = np.concatenate((y_train, y_test), axis=0)
label_encoder.fit(all_labels)

LabelEncoder()

In [ ]:
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train_encoded, num_classes)
y_test_categorical = to_categorical(y_test_encoded, num_classes)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to create an improved CNN model
def create_improved_model(input_shape=(48, 48, 1), num_classes=7):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Fourth convolutional block
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Global Average Pooling layer
    model.add(GlobalAveragePooling2D())

    # Fully connected layers
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Example of using ImageDataGenerator for data augmentation
def create_data_generator():
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    return datagen

In [ ]:
from tensorflow.keras.optimizers import Adam

# Create the model
model = create_improved_model()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generator
train_datagen = create_data_generator()

In [ ]:
# Train the model
model.fit(train_datagen.flow(X_train, y_train_categorical, batch_size=32), epochs=50)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 191s 208ms/step - accuracy: 0.2332 - loss: 1.9968
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 198s 204ms/step - accuracy: 0.3384 - loss: 1.6524
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 183s 203ms/step - accuracy: 0.3982 - loss: 1.5553
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 184s 205ms/step - accuracy: 0.4268 - loss: 1.4869
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 201s 204ms/step - accuracy: 0.4354 - loss: 1.4542
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 182s 203ms/step - accuracy: 0.4667 - loss: 1.4065
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 183s 204ms/step - accuracy: 0.4794 - loss: 1.3764
Epoch 8/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 184s 204ms/step - accuracy: 0.4874 - loss: 1.3550
Epoch 9/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 202s 205ms/step - accuracy: 0.4875 - loss: 1.3408
Epoch 10/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 203s 206ms/step - accuracy: 0.4974 - loss: 1.3134
Epoch 11/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 203s 207ms/step - accuracy: 0.5045 - loss: 1.3126
Epoch 12/50
898/898

In [ ]:
from tensorflow.keras.models import load_model

# Assuming 'model' is your trained Keras model
model.save('my_model.h5')  # Save the model in HDF5 format

# To load the model later
loaded_model = load_model('my_model.h5')

In [ ]:
# Evaluate the model
loss, accuracy = loaded_model.evaluate(X_test, y_test_categorical)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.5913 - loss: 1.2517
Test Accuracy: 51.66%


In [ ]:
# Generate classification report
y_pred = loaded_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))

113/113 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step
              precision    recall  f1-score   support

       Angry       0.51      0.44      0.47       491
     Disgust       0.65      0.31      0.42        55
        Fear       0.40      0.25      0.31       528
       Happy       0.66      0.89      0.76       879
     Neutral       0.26      0.25      0.26       626
         Sad       0.49      0.38      0.43       594
    Surprise       0.57      0.77      0.66       416

    accuracy                           0.52      3589
   macro avg       0.51      0.47      0.47      3589
weighted avg       0.49      0.52      0.49      3589



In [ ]:
# Function to detect and recognize emotions in a video using Haar Cascade
def detect_and_recognize_emotions(video_path):
    cap = cv2.VideoCapture(video_path)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize a dictionary to hold emotion counts
    emotion_counts = {emotion: 0 for emotion in label_encoder.classes_}
    total_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_image = gray_frame[y:y+h, x:x+w]
            face_image = cv2.resize(face_image, (48, 48))
            face_image = face_image.astype('float32') / 255.0
            face_image = face_image.reshape(-1, 48, 48, 1)

            emotion_prediction = loaded_model.predict(face_image)
            emotion_probabilities = emotion_prediction[0] * 100  # Convert to percentage

            # Update emotion counts
            for i, emotion in enumerate(label_encoder.classes_):
                emotion_counts[emotion] += emotion_probabilities[i]

        total_frames += 1

    cap.release()

    # Calculate average emotion percentages
    emotion_percentages = {emotion: (count / total_frames) for emotion, count in emotion_counts.items()}

    # Prepare the output list
    output = []
    for emotion, percentage in emotion_percentages.items():
        output.append(f"{emotion}: {percentage:.2f}%")

    # Specifically print depression percentage if it's in the dataset
    if 'sad' in emotion_percentages:  # Assuming 'sad' represents depression
        output.append(f"Depression Percentage: {emotion_percentages['sad']:.2f}%")

    return output

def draw_emotion_graph(emotion_percentages):
    emotions = list(emotion_percentages.keys())
    percentages = list(emotion_percentages.values())

    plt.figure(figsize=(10, 6))
    plt.bar(emotions, percentages, color='skyblue')
    plt.xlabel('Emotions')
    plt.ylabel('Percentage (%)')
    plt.title('Emotion Recognition from Video')
    plt.xticks(rotation=45)
    plt.ylim(0, 100)  # Set y-axis limit from 0 to 100%

    # Adding percentage labels on top of the bars
    for i, percentage in enumerate(percentages):
        plt.text(i, percentage + 1, f"{percentage:.2f}%", ha='center')

    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage
video_path = 'video2.mp4'
emotion_results = detect_and_recognize_emotions(video_path)
for result in emotion_results:
    print(result)

# Draw Graph of emotions
draw_emotion_graph(emotion_results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━

AttributeError: 'list' object has no attribute 'keys'